In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

In [2]:
def gather_data():
    """
    Description: This function gathers events data from "/home/workspace/event_data/" 
    directory. Gathered data saved into the DataFrames. Logs entries associated with 
    "NextSong" are selected for further transformations. 
    
    Arguments:
        None

    Returns:
        songs_lib_raw: is the DataFrame that containes songs play history.
    """

    filepath = os.getcwd() + '/event_data'
    csv_pattern = os.path.join(filepath,'*.csv')
    file_list = sorted(glob.glob(csv_pattern))
    
    dfs = []
    for file in file_list:
        data = pd.read_csv(file, encoding = 'utf8')
        dfs.append(data)
    df_playlogs = pd.concat(dfs, ignore_index=True) # (8056, 18)
    songs_lib_raw_ = df_playlogs.loc[df_playlogs["page"] == "NextSong"].reset_index(drop=True) # (6820, 17)
    songs_lib_raw = songs_lib_raw_[['artist','firstName','gender','itemInSession','lastName',
                                    'length','level','location','sessionId','song','userId']] #(6820, 11)
    return songs_lib_raw


In [3]:
songs_lib_raw = gather_data()
songs_lib_raw.head()

,artist,firstName,gender,itemInSession,lastName,length,level,location,sessionId,song,userId
0,Des'ree,Kaylee,F,1,Summers,246.30812,free,"Phoenix-Mesa-Scottsdale, AZ",139,You Gotta Be,8.0
1,Mr Oizo,Kaylee,F,3,Summers,144.03873,free,"Phoenix-Mesa-Scottsdale, AZ",139,Flat 55,8.0
2,Tamba Trio,Kaylee,F,4,Summers,177.18812,free,"Phoenix-Mesa-Scottsdale, AZ",139,Quem Quiser Encontrar O Amor,8.0
3,The Mars Volta,Kaylee,F,5,Summers,380.42077,free,"Phoenix-Mesa-Scottsdale, AZ",139,Eriatarka,8.0
4,Infected Mushroom,Kaylee,F,6,Summers,440.26730,free,"Phoenix-Mesa-Scottsdale, AZ",139,Becoming Insane,8.0


In [4]:
# establish connection to a Cassandra instance
from cassandra.cluster import Cluster
try: 
    cluster = Cluster(['127.0.0.1']) 
    session = cluster.connect()
except Exception as e:
    print(e)
    
# create the KEYSPACE udacity
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)
except Exception as e:
    print(e)
    
# Set KEYSPACE to the keyspace udacity
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

The PRIMARY KEY in the table **song_library** consists of two fields sessionId (Partition key) and itemInSession (Clustering key). Partitioning is done by **sessionId**. Withing that partition the rows are ordered by the **itemInSession**.

In [5]:
%%time
# Create table "song_library" and insert the values
query = "CREATE TABLE IF NOT EXISTS song_library " 
query = query + "(sessionId int, itemInSession int, artist text, song text, length decimal, PRIMARY KEY (sessionId, itemInSession))"
try:
    session.execute(query)
except Exception as e:
    print(e)                    

for index, row in songs_lib_raw.iterrows():
    query = "INSERT INTO song_library (sessionId, itemInSession, artist, song, length)"
    query = query + "VALUES (%s, %s, %s, %s, %s)"
    session.execute(query, (int(row["sessionId"]), int(row["itemInSession"]), row["artist"], row["song"], float(row["length"])))

CPU times: user 6.72 s, sys: 909 ms, total: 7.62 s
Wall time: 12.7 s


In [6]:
## Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4 

query = "SELECT artist, song, length FROM song_library WHERE sessionId=338 AND itemInSession=4"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist, row.song, row.length)

Faithless Music Matters (Mark Knight Dub) 495.3073


The PRIMARY KEY in the table **song_playlist_session**  consists of composite partition keys (**userId** and **sessionId**), and Clustering key **itemInSession**. Partitioning is done by **userId** and **sessionId**. Withing the partition rows are ordered by **itemInSession**.

In [7]:
%%time
# Create table "song_playlist_session" and insert the values
query = "CREATE TABLE IF NOT EXISTS song_playlist_session "
query = query + "(userId int, sessionId int, itemInSession int, artist text, song text, firstName text, lastName text, PRIMARY KEY ((userId, sessionId), itemInSession))"
try:
    session.execute(query)
except Exception as e:
    print(e)                    

for index, row in songs_lib_raw.iterrows():
    query = "INSERT INTO song_playlist_session (userId, sessionId, itemInSession, artist, song, firstName, lastName)"
    query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
    session.execute(query, (int(row["userId"]), int(row["sessionId"]), int(row["itemInSession"]), row["artist"], row["song"], row["firstName"], row["lastName"]))

## Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182
query = "SELECT artist, song, firstName, lastName FROM song_playlist_session WHERE userId=10 AND sessionId=182"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist, row.song, row.firstname, row.lastname)

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz
CPU times: user 7.55 s, sys: 875 ms, total: 8.43 s
Wall time: 14 s


The PRIMARY KEY in the table **songs_by_users** consists of two fields song (Partition key) and userId (Clustering key). Partitioning is done by **song**. Withing that partition the rows are ordered by the **userId**.

In [8]:
%%time
# Create table "songs_by_users" and insert the values
query = "CREATE TABLE IF NOT EXISTS songs_by_users " 
query = query + "(song text, userId int, firstName text, lastName text, PRIMARY KEY (song, userId))"
try:
    session.execute(query)
except Exception as e:
    print(e)                    

for index, row in songs_lib_raw.iterrows():
    query = "INSERT INTO songs_by_users (song, userId, firstName, lastName)"
    query = query + "VALUES (%s, %s, %s, %s)"
    session.execute(query, (row["song"], int(row["userId"]), row["firstName"], row["lastName"]))

# Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
query = "SELECT firstName, lastName FROM songs_by_users WHERE song='All Hands Against His Own' "
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.firstname, row.lastname)


Jacqueline Lynch
Tegan Levine
Sara Johnson
CPU times: user 6.47 s, sys: 890 ms, total: 7.36 s
Wall time: 12.3 s


In [9]:
# Drop the table before closing out the sessions
query = "drop table song_library"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

query = "drop table song_playlist_session"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

query = "drop table songs_by_users"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

In [10]:
# Close the session and cluster connection
session.shutdown()
cluster.shutdown()